In [1]:
from chemdataextractor.doc import Document

text = "Ti-6Al-4V and CuSO4·5H2O were analyzed."
doc = Document(text)
print([cem.text for cem in doc.cems])


['CuSO4·5H2O']


In [2]:
import re
from chemdataextractor.doc import Document

def extract_all_formulas(text: str):
    # use chemDataExtractor for standard chemicals
    cde_formulas = {cem.text for cem in Document(text).cems}

    # regex for alloy-style or hyphenated metals
    alloy_pattern = r'\b[A-Z][a-z]?(?:[-–]\d*[A-Z][a-z]?\d*)+\b'

    # regex for chained or hydrated chemical formulas
    formula_pattern = r'(?:[A-Z][a-z]?\d*)+(?:·\d*(?:[A-Z][a-z]?\d*)+)?'

    # combine all results into one unique set
    alloys = set(re.findall(alloy_pattern, text))
    chains = set(re.findall(formula_pattern, text))

    return sorted(cde_formulas | alloys | chains)

# example test
text = "Ti-6Al-4V and CuSO4·5H2O were analyzed in Fe-Cr-Ni alloys."
extract_all_formulas(text)


['Al', 'Cr', 'CuSO4·5H2O', 'Fe', 'Fe-Cr-Ni', 'Ni', 'Ti', 'Ti-6Al-4V', 'V']